# LangChain RAG
## RAG (Retreival Augumented Generation)
LLM에 학습되지 않은 정보를 물어보면, 대답을 못하거나, Hallucination(환각현상:헛소리)이 발생한다. 


![only llm](imgs/api.png)

사용자의 질문과 관계있는 정보를 검색해서 답변을 생성할 수 있게 해주면, LLM의 단점을 줄이고, 한계를 극복할 수 있다. 

답변 하기 전에 관련있는 컨닝페이퍼를 몇개 가져와서 읽어보고 답변하는 방식이라고 이해하면 된다. 

![rag](imgs/rag.png)

- 만약 GPT에게 몇주전 한국에서 화재가 되었던 "민희진 하이브 논란"에 대해 물어보면 답을 하지 못할 것이다. 
- RAG를 이용한다면, 관련 기사를 제공하고 llm이 답변할 때, 그 기사를 참고해서 답변할 수 있을 것이다. 

## Chunking & Embedding

![embedding](imgs/embedding.png)

### Chunk
- 그러나, 문서의 분량이 엄청나다면 그 문서를 통채로 llm에 넘길 수는 없다. 
- llm이 처리할 수 있는 문서 길이의 한계가 있기 때문이다. (사람도 마찬가지)
- 그래서 작은 단위로 문서를 쪼개 놓고, 질문과 관련있어 보이는 조각을 llm으로 가져와서 답변을 만든다. 
  - 큰 문서를 작은 단위로 쪼개는 작업 --> `chunking`
  - 처리할 수 있을 정도의 크기로 짧게 자른 단위 --> `chunk`
- chunking하는 여러가지 방법이 있음

### Embedding
- 그렇다면, 수백, 수천개의 `chunk` 중에서 질문 내용과 가장 유사한 내용을 담고 있는 `chunk`를 어떻게 검색할 수 있을까? 
- 각 `chunk`를 벡터 형태로 변환하여 저장
   - 벡터로 변환하는 방법, 모델은 여러가지가 있음 
   - openAI에서도 API 형태로 제공


In [1]:
from langchain_openai import ChatOpenAI
from env import OPENAI_API_KEY

chat = ChatOpenAI(model="gpt-4", temperature=0.2, api_key=OPENAI_API_KEY)

## Chunking 

In [2]:
!pip install beautifulsoup4

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://namu.wiki/w/%EB%AF%BC%ED%9D%AC%EC%A7%84-HYBE%20%EA%B0%84%20ADOR%20%EA%B2%BD%EC%98%81%EA%B6%8C%20%EB%B6%84%EC%9F%81")
data = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [5]:
for split in all_splits:
    print(split)
    print('----------')

page_content='민희진-HYBE 간 ADOR 경영권 분쟁 - 나무위키  최근 변경  최근 토론  특수 기능              민희진-HYBE 간 ADOR 경영권 분쟁' metadata={'source': 'https://namu.wiki/w/%EB%AF%BC%ED%9D%AC%EC%A7%84-HYBE%20%EA%B0%84%20ADOR%20%EA%B2%BD%EC%98%81%EA%B6%8C%20%EB%B6%84%EC%9F%81', 'title': '민희진-HYBE 간 ADOR 경영권 분쟁 - 나무위키', 'language': 'No language found.'}
----------
page_content='최근 수정 시각: 2024-05-11 12:51:58  84   편집     편집  편집 권한이 부족합니다. 가입한지 15일 지난 사용자(이)여야 합니다. 해당 문서의 ACL 탭을 확인하시기 바랍니다.   닫기  토론  역사         분류 HYBE/논란 및 사건 사고ADOR2024년/사건사고인터넷 밈/정치 및 사회/대한민국      \xa0 가입 후 15일이 지나야 편집 가능한 문서입니다. (~ KST ) \xa0 해당 분쟁 과정에서 일어난 HYBE의 단월드 연관 의혹에 대한 내용은 HYBE-단월드 연관설 문서를의 번 문단을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번 문단을의 번 문단을의  부분을의  부분을, 에 대한 내용은  문서를의 번

In [6]:
print(type(all_splits[0]))

<class 'langchain_core.documents.base.Document'>


## Embedding

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


vectorstore = Chroma.from_documents(
    documents=all_splits, 
    embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY)
)

## Retreiver

In [8]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=3)

docs = retriever.invoke("민희진의 직업은 무엇인가?")

for d in docs:
    print(d)
    print('------')

page_content='오후 민희진 대표 측이 내놓은 재반박문에 대해 하이브는 다시 한번 공식입장을 냈다.HYBE 공식입장(2) 전문 [ 펼치기 · 접기 ]안녕하세요, 하이브입니다.10일 오후 민희진 대표 측이 내놓은 공식 입장에 대해 설명드립니다.민희진 대표는 ‘역량이 높은 ‘내부’ 인재가 올린 성과 보상을 ‘외부’로부터 수취하는 것이 정당하다’는 황당한 궤변을 늘어놓고 있습니다. 이는 관행이 아니라 불법입니다.유연한 보상체계가 필요하다면 회사가 수령하고 다시 인센티브로 정당하게 지급해야 합니다. 민 대표는 경영권 탈취시도를 ‘사담’이라고 치부하더니 이번엔 불법을 ‘관행’이라고 강변하고 있습니다.민 대표는 어도어 경영진과의 대화에서 해당 팀장의 비위에 대해 "광고 피를 혼자 먹지 않냐. 어시(어시스트 직원)들은 안 받으면서 일하고, 이거 문제라고 생각하지 않냐. 사실 처음에 허락했을 때는 우리도 미처 이 문제에 대해 심각하게 생각하지 못했다"라고 말했습니다.일은 회사 구성원들이 하고, 이익은 팀장이 사적으로 챙기는 것이 문제가 있음을 민 대표도 충분히 인지하고 있음을 드러내는 대목입니다.애초 이 건은 올해 2월 해당 팀장의 인센티브가 0원이 책정된 것을 의아하게 생각한 하이브 HR팀이 어도어에 문의하면서 인지됐고, 당시 어도어 측은 “관행이다, 개선하려 한다”고 설명했을 뿐 아무런 소명자료도 제출하지 않았습니다. 당사는 이후 감사 과정에서 발견한 정황 증거를 확인한 뒤에 심각한 비위 행위임을 파악할 수 있었습니다.모 매체에 게재된 어도어 팀장의 인터뷰와 관련해서도 설명 드립니다.9일 저녁 진행된 감사는 전혀 강압적이지 않은 분위기에서 진행됐고, 해당 팀장도 자발적인 협조 의사를 밝혀 자택에 보관 중인 노트북 제출까지 진행됐습니다.당사는 신원이 철저히 보호돼야 할 팀장급 직원을 앞세우는 민 대표의 행태에 다시 한번 강력한 유감을 표합니다.불법행위가 의심되는 상황에서 회사는 사실관계를 확인해야 할 정당한 권한을 갖습니다. 오히려, 불법행위에 관여한 당사자에게 

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.chains.combine_documents import create_stuff_documents_chain

chat = ChatOpenAI(model="gpt-4", api_key=OPENAI_API_KEY)

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [10]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("민희진은 누구와 분쟁을 하고 있나? 무엇이 문제인가?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

"민희진은 하이브(HYBE)와 분쟁을 하고 있습니다. 이 분쟁의 주요 문제점은 민희진 대표가 '역량이 높은 ‘내부’ 인재가 올린 성과 보상을 ‘외부’로부터 수취하는 것이 정당하다'는 주장을 제기하고 있는 것입니다. 하이브는 이를 불법으로 간주하고 있으며, 민희진이 이를 '관행'이라고 강변하는 것에 반발하고 있습니다. 또한, 민희진이 어도어 경영진과의 대화에서 해당 팀장의 비위에 대해 부당하게 이익을 취하는 것에 대한 문제 인식을 보여주었고, 이는 회사 구성원들이 일하고, 이익은 팀장이 사적으로 챙기는 것이 문제라는 것을 인지하고 있다는 점을 드러냈습니다. 또한, 하이브는 민희진이 경영권 탈취시도를 하는 등의 불법행위에 대해 강력하게 반발하고 있습니다."